In [ ]:
#########################################################################
# LIBRERIAS NECESARIAS
#########################################################################
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import LoggerWrapper
import torch

import tensorflow as tf
from stable_baselines3 import PPO
from tensorflow.keras.layers import concatenate
import keras
import matplotlib.pyplot as plt
from keras import layers
import copy
from sklearn.model_selection import train_test_split
import csv

import os
from stable_baselines3.common.vec_env import DummyVecEnv

In [ ]:
############################################################################
# Variables globales
###########################################################################
BATCH_SIZE = 28032
EPOCHS =100
EPISODES = 1
EPISODES_EVALUATE_G=5
TOTAL_TIMESTEPS_PPO_GENERATOR=250

In [ ]:
# Definimos el entorno Warehouse
env = gym.make('Eplus-warehouse-hot-discrete-v1')
env = LoggerWrapper(env)

# Obtenemos el espacio de observaciones y el espacio de acciones del entorno env
ob_space = env.observation_space
ac_space = env.action_space

# Mostramos el número de observaciones y de acciones del entorno 5Zone
print('\n Nº de observaciones: ', ob_space.shape[0])
print('\n Nº de acciones: ', ac_space.n)

# Disriminador

## Red neuronal del Discriminador

In [ ]:
################################################################################################################################################
# Red neuronal del Discriminador
################################################################################################################################################

# Input: secuencias [s,a] reales o falsas, de longitud 2*ob_space.shape[0] + ac_space.n+1. 
# Output: probabilidad de real o falso perteneciente al intervalo [0,1]
discriminator_net = keras.Sequential(
    [
        keras.Input(shape=(None, 2*ob_space.shape[0] + ac_space.n+1)),
        layers.Dense(units=17, activation=tf.nn.relu, name='layer1'),
        layers.Dense(units=17, activation=tf.nn.relu, name='layer2'),
        layers.Dense(units=17, activation=tf.nn.relu, name='layer3'),
        layers.Dense(units=1, activation=tf.sigmoid, name='prob'),

    ],
    name="discriminator_net"

)
discriminator_net.summary()

## Función de pérdida del Discriminador 

In [ ]:
#########################################################################################################
# Función de pérdida del Discriminador
#########################################################################################################

# prob1=> output de la red neuronal del Discriminador cuando recibe como entrada una secuencia REAL [s,a] de la base de datos 
# prob2=> output de la red neuronal del Discriminador cuando recibe como entrada una secuencia FALSA [s,a] 
def loss_fn_D(prob1, prob2):

    # Esperanza del logaritmo de la D(x)=salida de la red neuronal cuando x=entrada REAL
    loss_expert = tf.reduce_mean(tf.math.log(tf.clip_by_value(prob1, 0.01, 1)))
    
    # Esperanza del logaritmo de 1-D(x) donde D(x)=salida de la red neuronal cuando x=entrada FALSA
    loss_agent = tf.reduce_mean(tf.math.log(tf.clip_by_value(1 - prob2, 0.01,1)))
    
    loss_expert = tf.cast(loss_expert, dtype=tf.float32)
    loss_agent = tf.cast(loss_agent, dtype=tf.float32)
    
    loss = loss_expert + loss_agent
    
    loss = -loss

    return loss

## Clase del Discriminador

In [ ]:
class Discriminator:
    def __init__(self,env,discriminator_net,expert_s,expert_a,agent_s,agent_a):

        # -Red neuronal del Discriminador
        self.discriminator_net = discriminator_net

        # 1) Secuencia experta: [s,a]
        self.expert_s = expert_s
        self.expert_a = expert_a
        expert_a_one_hot = tf.one_hot(self.expert_a, depth=env.action_space.n)
        # Añadimos ruido para estabilizar el entrenamiento
        expert_a_one_hot += tf.random.normal(tf.shape(expert_a_one_hot),mean=0.2,stddev=0.1,dtype=tf.float32) / 1.2
        # expert_s_a=>secuencia experta=>[s,a]
        self.expert_s_a = tf.concat([self.expert_s, expert_a_one_hot], axis=1)

        # 2) Secuencia agente:  [s,a]
        self.agent_s = agent_s
        self.agent_a = agent_a
        agent_a_one_hot = tf.one_hot(self.agent_a, depth=env.action_space.n)
        agent_a_one_hot += tf.random.normal(tf.shape(agent_a_one_hot),mean=0.2,stddev=0.1,dtype=tf.float32) / 1.2
        # agent_s_a=>secuencia agente=>[s,a]
        self.agent_s_a = tf.concat([self.agent_s, agent_a_one_hot], axis=1)

        # Calculamos la salida de la red para [s,a] del experto y [s,a] del agente 

        # -Salida de la red neuronal Discriminador para [s,a] expertos(verdaderos)
        self.prob_expert = self.discriminator_net(self.expert_s_a)

        # -Salida  de la red neuronal Discrimiinador para [s,a] Agente(falsos)
        self.prob_agent = self.discriminator_net(self.agent_s_a)

        # -Recompensa obtenida cuando el Agente realiza [s,a] falsas
        self.rewards = tf.math.log(tf.clip_by_value(self.prob_agent, 1e-10, 1))

    def getNet(self):
        return self.discriminator_net

    def getAgent_S_A(self):
        return self.agent_s_a

    def getExpert_S_A(self):
        return self.expert_s_a

    def getProb(self):
        return self.prob_expert, self.prob_agent

    def getRewards(self):
        return self.rewards

# Generador

## Redes neuronales del Generador

In [ ]:
# Red neuronal del Generador donde se producen acciones
# Input: estados, listas de tamaño 20, s=[s1,s2,s3,....,s23]
# Output: acciones, listas de tamaño 10, a=[a1,a2,....,a10]
generator_net_Act = keras.Sequential(
    [
        keras.Input(shape=(None, ob_space.shape[0])),
        layers.Dense(units=20, activation=tf.tanh, name='layer1'),
        layers.Dense(units=20, activation=tf.tanh, name='layer2'),
        layers.Dense(units=15, activation=tf.tanh, name='layer3'),
        layers.Dense(units=ac_space.n, activation=tf.nn.softmax, name='layer4')

    ],
    name="generator_net_Act"
)

generator_net_Act.summary()

In [ ]:
# Red neuronal del Generador donde se producen v_pred
# Input: estados, listas de tamaño 20, s=[s1,s2,s3,....,s17]
# Output: v_pred, listas de tamaño 1, v_pred(s)

generator_net_v_preds = keras.Sequential(
    [
        keras.Input(shape=(None, ob_space.shape[0])),
        layers.Dense(units=20, activation=tf.tanh, name='layer1'),
        layers.Dense(units=20, activation=tf.tanh, name='layer2'),
        layers.Dense(units=1, activation=None, name='layer3'),
    ],
    name="generator_v_preds"
)

generator_net_v_preds.summary()

## Función de pérdida del Generador 

In [ ]:
# Función de pérdida del Generador: función objetivo de PPO "clipped surrogated" 
def loss_fn_ppo(act_probs,act_probs_old,gaes,clip_value=0.2):
    ratios = tf.exp(tf.math.log(tf.clip_by_value(act_probs, 1e-10, 1.0))
                    - tf.math.log(tf.clip_by_value(act_probs_old, 1e-10, 1.0)))

    clipped_ratios = tf.clip_by_value(ratios,clip_value_min=1 -clip_value,clip_value_max=1 +clip_value)
    loss_clip = tf.minimum( tf.multiply(gaes, ratios), tf.multiply(gaes, clipped_ratios))
    loss_clip = tf.reduce_mean(loss_clip)
    
    loss = -loss_clip
    tf.summary.scalar('total', loss)

    return loss

## Clase del Generador 

In [ ]:
#################################################################################################################
# Clase del GENERADOR: política con su optimizador PPO
################################################################################################################

# Observesé que cada generador implementa una política distinta, por tanto, se ha decidido llamar a la clase Policy_net en lugar de generator

class Policy_net:
    def __init__(self, name: str, env, obs):
        """
        name: string
        env: gym env
        obs:
        """

        # -Entorno
        self.env = env
        env.reset()

        # -Modelo PPO: algoritmo de Optimización de Política Proximal  
        self.model = PPO(policy="MlpPolicy", env=env, verbose=0)

        self.model.learn(total_timesteps=TOTAL_TIMESTEPS_PPO_GENERATOR)  # 25000)

        # -Observación inicial a partir de la cual se crean las acciones iniciales haciendo uso de las redes neuronales del generador
        self.obs = np.reshape(np.array(obs), (1, ob_space.shape[0]))

        # Utilizamos las dos redes neuronales que hemos creado : generator_net_Act y generator_net_v_preds
        # V_pred=>recompensa media de que un agente ejecute una acción

        # -Acción inicial generada con red neuronal y v_pred con red neuronal
        self.act_probs = generator_net_Act(self.obs)
        self.v_preds = generator_net_v_preds(self.obs)

        # -Accion estocástica inicial
        self.act_stochastic = tf.random.categorical( tf.math.log(self.act_probs), num_samples=1)

        # -Acción determinística inicial
        self.act_deterministic = tf.argmax(self.act_probs, axis=1)

    # Para cada estado obs me dice la acción que el agente va a ejecutar sobre el entorno junto con v_pred
    # La elección de la acción puede ser estocástica o determinística

    def act(self, stochastic=True):
        if stochastic:
            return self.act_stochastic, self.v_preds
        else:
            return self.act_deterministic, self.v_preds

    def get_action_prob(self):
        return self.act_probs

    def get_v_preds(self):
        return self.v_preds

    def get_obs(self):
        return self.obs

    def get_model(self):
        return self.model

    def get_trainable_variables(self):
        return self.model.get_parameters()

    # Generar [s,a] falsos
    def generate_fakes(self):

        ob_space = env.observation_space
        reward = 0
        success_num = 0

        # Por cada episodio
        for iteration in range(EPISODES):
            # Inicializo todas las variables
            observations = []
            actions = []
            rewards = []
            run_policy_steps = 0

            truncated = False
            terminated = False

            # La primera acción de cada episodio se crea con la red neuronal

            obs,_ = env.reset()

            Old_Policy = Policy_net('old_policy', env, obs=[obs])

            act, _ = Old_Policy.act(stochastic=True)

            # Convertir de tensor a array
            if type(act)=='Tensor':
                # Crear una sesión de TensorFlow
                sess = tf.compat.v1.Session()

                # Evaluar el tensor dentro de la sesión y obtener el resultado como un objeto NumPy ndarray
                act = sess.run(act)

                # Cerrar la sesión
                sess.close()

            if isinstance(act, tf.Tensor):
                act = act.numpy()

            elif isinstance(act, np.ndarray):
                act = act

            action = int(act)

            next_obs, reward, terminated, truncated, _ = env.step(action)

            # --Actualización de variables: ojo no introduzco el estado y accion inicial, solo introduzco los de PPO
            observations.append(next_obs)  # S_0

            # tenemos una política entrenada
            Policy = Policy_net('policy', env, obs=[next_obs])

            # Por cada steps en cada episodio, mientras no se llegue a un
            # estado terminal o un estado malo
            while terminated != True and truncated != True:
                # --Aumentar el numero de steps
                run_policy_steps += 1

                # --Política para ver la acción asociada al estado
                # Las observaciones son un de la forma [[s_0,s_1,s_2,....]] por eso su tamaño es (1,n)
                action, _ = Policy.get_model().predict(next_obs)

                action = int(action)

                # --Muevo al Agente al siguiente estado
                next_obs, reward, terminated, truncated, _ = env.step(action)

                # --Actualización de variables
                actions.append(action)  # A_i-1
                rewards.append(reward)  # R_i-1

                # --Si llegamos a un estado final, el juego ha finalizado!!!
                # --Se configura el tablero de nuevo
                if terminated or truncated:
                    obs,_= env.reset()
                    reward = -1
                    break
                else:
                    observations.append(next_obs)  # S_i
                    self.obs = next_obs

            # Ver si el episodio ha obtendo una recompensa total igual o superior a 195
            if sum(rewards) >= 195:
                success_num += 1
                if success_num >= 100:
                    break
            else:
                success_num = 0

        observations = np.reshape(observations,newshape=[-1] + list(ob_space.shape))

        actions = np.array(actions).astype(dtype=np.int32)

        return observations, actions, rewards, Old_Policy, Policy



In [ ]:
##########################################################################################################
# Clase PPOTrain
##########################################################################################################
# Tenemos dos politica theta_i y theta_i+1
# Almacenamos dos políticas Policy_net(cada una de ella con su PPO) y calculamos el valor gaes a partir de valores gamma, clip_value, c_1, c_2
# Realizamos aqui el entrenamiento, cálculo de gradiente y función de pérdida del PPO para después usarlo en el generador de la GAN
class PPOTrain:
     def __init__(self,Policy,Old_Policy,obs,actions,rewards,gamma=0.95):
        """
        arg:
            Policy
            Old_Policy
            gamma
            clip_value
            c_1 parámetro para la diferencia de valores
            c_2 parámetro para el bonus de entropía
        """
        self.Policy = Policy
        self.Old_Policy = Old_Policy
        self.gamma = gamma
        self.obs = obs

        self.pi_trainable = self.Policy.get_trainable_variables()
        self.old_pi_trainable = self.Old_Policy.get_trainable_variables()

        policy_dict_ = self.pi_trainable["policy"]
        old_policy_dict_ = self.old_pi_trainable["policy"]

        self.pi = []
        if "policy" in self.pi_trainable and  "policy" in self.old_pi_trainable:
            for param_name, param_value in policy_dict_.items():
                # Elimino los pesos que hay en old_policy
                del old_policy_dict_[param_name]
                # Introduzco los pesos de old_policy en policy
                old_policy_dict_[param_name] = param_value
                self.pi.append(param_value)
        else:
            print("No se encontró la política con el nombre: policy")

        # Le asignamos old_pi_trainable=pi_trainable ya que ajustaremos unos
        # nuevos pi_trainable
        self.actions = actions
        self.rewards = rewards
        self.v_preds = self.Old_Policy.get_v_preds()
        self.v_preds_next = self.Policy.get_v_preds()
        
        #  La estimación de la función ventaja GAES, ver paper PPO
        self.gaes = self.get_gaes(self.rewards,self.v_preds,self.v_preds_next)

        act_probs = self.Policy.get_action_prob()
        act_probs_old = self.Old_Policy.get_action_prob()

        # la probabilidad de las acciones del agente cuando toma la actual política
        act_probs = act_probs * tf.one_hot(indices=self.actions, depth=act_probs.shape[1])
        self.act_probs = tf.reduce_sum(act_probs, axis=1)

        # la probabilidad de las acciones del agente cuando toma la antigua política
        act_probs_old = act_probs_old * tf.one_hot(indices=self.actions, depth=act_probs_old.shape[1])
        self.act_probs_old = tf.reduce_sum(act_probs_old, axis=1)

        self.loss = loss_fn_ppo( self.act_probs, self.act_probs_old, self.gaes)

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

    def loss_fn_G(self):
        return loss_fn_ppo(self.act_probs, self.act_probs_old, self.gaes)

    def get_pi_trainable(self):
        return self.pi

    def get_optimizer(self):
        return self.optimizer

    def get_OldPolicy(self):
        return self.Old_Policy

    def get_Policy(self):
        return self.Policy

    def get_gaes(self, rewards, v_preds, v_preds_next):
        deltas = [r_t + self.gamma *v_next - v for r_t, v_next, v in zip(rewards,v_preds_next,v_preds)]
        # calcular la estimación generative advantage (lambda = 1), ver ppo paper eq(11)
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(gaes) -1)):  # es T-1, donde T es time step con el que se ejecuta la política
            gaes[t] = gaes[t] + self.gamma * gaes[t + 1]
        return gaes


# GAIL 

In [ ]:
##########################################################################
# CLASE GAIL
##########################################################################

class GAN(keras.Model):
    # Constructor
    def __init__(self, discriminator, generator):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.i=0
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    # Compila el modelo GAN inicializando los optimizadores y la función de pérdida del modelo GAN
    def compile(self, d_optimizer, loss_fn_D):
        super(GAN, self).compile(run_eagerly=True)
        self.d_optimizer = d_optimizer
        self.loss_fn_D = loss_fn_D

    # Devuelve las métricas obtenidas con el generador y discriminador
    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    # Evaluación del Discriminador
    def evaluate_D(self, X_test):
    
        len_real = X_test.shape[0]

        # Generamos datos falsos dataset_gen = [s,a]
        generate_observations, generate_actions, rewards, Old_Policy, Policy = self.generator.generate_fakes()

        generate_a_one_hot = np.eye(env.action_space.n)[generate_actions]

        dataset_gen = np.concatenate([generate_observations, generate_a_one_hot], axis=1)


        dataset_gen = dataset_gen.astype('float32')
        dataset_gen[-ac_space.n:] = dataset_gen[-ac_space.n:].astype('int32')
        dataset_gen= tf.convert_to_tensor(dataset_gen)
        
        X_test = X_test.astype('float32')
        X_test[-ac_space.n:] = X_test[-ac_space.n:].astype('int32')

    
        len_fakes = dataset_gen.shape[0]


        # Compilamos el discriminador como CNN
        discriminator_net.compile( optimizer=tf.keras.optimizers.Adam( learning_rate=1e-5), loss=loss_fn_D, metrics=['accuracy'])

        #Evaluamos como CNN
        loss_real, acc_real = discriminator_net.evaluate(X_test, tf.ones((len_real,1)), batch_size=len_real, verbose=1)

        loss_fake, acc_fake =discriminator_net.evaluate(dataset_gen, tf.ones((len_fakes, 1)), batch_size=len_fakes, verbose=1)

        print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real * 100, acc_fake * 100))
        print('>Loss real: ')
        
        print(loss_real)
        print('>Loss fake: ')
        print(loss_fake)

    
    # Evaluación del generador

    def evaluate_G(self):
        # Definimos el entorno
        env = gym.make('Eplus-warehouse-hot-discrete-v1')
        env = LoggerWrapper(env)

        # NUESTRO OBJETIVO: Agente aprenda a tomar las acciones que maximicen
        # la recompensa

        # Lista donde amacenaremos la recompensa acumulada de cada episodio.
        rewards = []

        # Para cada episodio, el Agente se mueve por el Entorno mediante acciones hasta llegar a un estado final
        # siguiendo la política que se ha aprendido en el entrenamiento de la
        # GAN
        for episode in range(EPISODES_EVALUATE_G):
            truncated = False
            terminated = False
            R = 0.0
            reward = 0.0

            # Estado inicial del juego
            obs, _ = env.reset()

            # Interactuamos con el Entorno hasta que lleguemos a un estado
            # final
            while terminated != True and truncated != True:
                action, _ = self.generator.get_model().predict(obs)
                obs, reward, terminated, truncated, _ = env.step(int(action))

                # Incremento la recompensa del episodio i al haber ejecutado el step
                R += reward

            rewards.append(R)

            # Vemos para el episodio, su recompensa acumulada que es lo que se
            # trata de maximizar
            print("Episode  {} Total reward: {}".format(episode, R))

        # Cierro el entorno
        env.close()

        # Muestro las recompensas obtenidas en cada episodio
        indices = range(0, EPISODES_EVALUATE_G)
        plt.plot(indices, rewards)
        plt.show()

        return np.mean(rewards)
    
    def train_step(self, X_train):
        # 1) Generamos secuencias falsas [s,a]
        generate_observations, generate_actions, rewards, Old_Policy, Policy = self.generator.generate_fakes()

        generate_a_one_hot = np.eye(env.action_space.n)[generate_actions]

        if generate_observations.shape[0] == generate_a_one_hot.shape[0]:
            dataset_gen = np.concatenate([generate_observations, generate_a_one_hot], axis=1)
        else:
            generate_a_one_hot_resized = np.resize(generate_a_one_hot, generate_observations.shape)
            dataset_gen = np.concatenate([generate_observations, generate_a_one_hot_resized], axis=1)

    
        dataset_gen = dataset_gen.astype('float32')
        dataset_gen[-10:] = dataset_gen[-10:].astype(int)

        dataset_gen= tf.convert_to_tensor(dataset_gen)

        # 2) Obtenemos las secuencias reales [s,a] de los datos de entrenamiento
        X_train = tf.strings.to_number(X_train)
        X_train = tf.cast(X_train, dtype=tf.float32)

        combined_images = tf.concat([X_train, dataset_gen], axis=0)
        
        # 3) Longitud de datos pueden ser diferente en secuencias reales y secuencias falsas. 
        # Tener en cuenta que no tenemos la misma cantidad de datos verdaderos y falsos, por eso calculamos len_real y len_fakes
        # No podemos controlar la creación de x secuencias [s,a, s', r] ya que generaremos tantas secuencias como se necesiten para finalizar el juego
        len_fakes = dataset_gen.shape[0]
        len_real = X_train.shape[0] 

        # 4) Las etiquetas de las imagenes combinadas las tenemos que crear nosotros introduciendo algo de ruido con tf.random.uniform
        labels = tf.concat([tf.ones((len_real, 1)), tf.zeros((len_fakes, 1))],axis=0)

        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        ##############################################################################################################################################################
        # PASO 1:  ENTRENAMIENTO DEL DISCRIMINADOR 
        #############################################################################################################################################################

        # Entrenamiento del discriminador con las [s,a] del agente (falsas o sintéticas) y del experto (reales) combinadas, esto es,
        # le pasamos un conjunto que tiene tanto trayectorias reales como trayectorias falsas
        with tf.GradientTape() as tape:
            predictions = np.zeros((len_real + len_fakes, ob_space.shape[0] + ac_space.n))
            # Predicciones obtenidas con el discriminador
            predictions = self.discriminator.discriminator_net(combined_images)
            # Valor de la función de pérdida al comparar las predicciones con las etiquetas reales
            d_loss = self.loss_fn_D(labels, predictions)

        # Calculo del gradiente y actualización del gradiente
        grads = tape.gradient(d_loss, self.discriminator.getNet().trainable_weights)

        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.getNet().trainable_weights)
        )

        ################################################################################################################################################################
        # PASO 2: ENTRENAMIENTO DEL GENERADOR=POLÍTICA 
        ######################################################################################################################

        ppotrain = PPOTrain(Policy,Old_Policy,actions=generate_actions,rewards=rewards,obs=generate_observations[0])

        with tf.GradientTape() as tape:
            g_loss = ppotrain.loss_fn_G()

        g_loss = tf.cast(g_loss, dtype=tf.float32)

        #################################################################################################################################################################
        # PASO 3: Cada 10 épocas, ejecutamos al generador (agente) sobre el entorno. Esto es, llamamos al método evaluate_G.  
        #################################################################################################################################################################
        # Por cada 10 epoca quiero conocer los datos de recompensa
        if (self.i+1)%10==0:
          rewardMean=self.evaluate_G()
          print('\nEpoca:', self.i+1)
          print('\nRecompensa de Media:', rewardMean, '\n')

        self.i=self.i+1

        ############################################################################################################################################################

        # Actualización de métricas del discriminador y generador
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {"d_loss": self.d_loss_metric.result(),
                "g_loss": self.g_loss_metric.result()}



# Base de datos experta

In [ ]:
##########################################################################
# LECTURA DEL DATASET REAL
##########################################################################
with open('observations_Warehouse.csv', 'r') as archivo_csv:
    # Crear el lector CSV
    expert_observations = csv.reader(archivo_csv)
    with open('actions_Warehouse.csv', 'r') as archivo_csv:
        expert_actions = csv.reader(archivo_csv)

        # Construimos el dataset [s,a] reales y lo dividimos en training y test
        expert_actions = [int(item[0]) for item in list(expert_actions)][:]
        expert_a_one_hot = np.eye(env.action_space.n)[expert_actions]
        expert_observations = list(expert_observations)
        expert_observations = np.array(expert_observations)
        dataset = np.concatenate([expert_observations, expert_a_one_hot], axis=1)

        print('\n Tamanio del dataset: ', dataset.shape)
        print('\n Primera fila del dataset: ', dataset[0])



In [ ]:
#############################################################################
# DIVISIÓN TRAIN Y TEST
##############################################################################

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test = train_test_split(dataset, test_size=0.2, random_state=0)


# Imprime el número de elementos en el conjuntos de entrenamiento y prueba
print('Nº de (ESTADOS,ACCIONES) en el conjunto de entrenamiento:', len(X_train))
print('Nº de (ESTADOS,ACCIONES) en el conjunto de prueba:', len(X_test))

# EXPERIMENTACIÓN DE GAIL CON WAREHOUSE

## Definición de generador y discriminador 

In [ ]:
obs, info = env.reset(seed=1)

# Generador
generator = Policy_net('policy', env, obs)

# Generamos [s,a]=[observations,actions] falsas y las políticas theta_i=Old_Policy y theta_i+1=Policy
observations, actions, rewards, Old_Policy, Policy = generator.generate_fakes()

# Discriminador
discriminator = Discriminator(env,discriminator_net,expert_observations,expert_actions,observations,actions)


## Experimento 1: 


### Definición de Extended GAIL 

In [ ]:
gan = GAN(discriminator=discriminator, generator=generator)

### Compilación de  GAIL

In [ ]:
tf.config.run_functions_eagerly(True)

# Compilación 
gan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss_fn_D=loss_fn_D
)


### Entrenamiento de GAIL

In [ ]:
# Deshabilitar los mensajes de información de TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Deshabilitar los mensajes de información de OpenAI Gym
gym.logger.set_level(40)

# Entrenamiento GAIL
history = gan.fit(X_train,epochs=EPOCHS, batch_size=BATCH_SIZE)

### Evaluación de GAIL

In [ ]:
gan.evaluate_D(X_test)

# Análisis de Experimentos 

In [ ]:
# Librerias necesarias para la visualización
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Experimento 1

In [ ]:
x=[10,20,30,40,50,60,70,80,90,100]
y=[]

df = pd.DataFrame({'x': x, 'y': y})

### Gráfica de Recompensa Media/ Épocas

In [ ]:
plt.plot(x, y, label='Warehouse',color='green')

#Agregamos las etiquetas y añadimos una leyenda.
plt.xlabel('Épocas')
plt.ylabel('Recompensa media')
plt.title("Warehouse")
# plt.legend()
plt.savefig('grafica_Warehouse_Extended_Gail.png')
plt.show()

### Estimación estadística: Modelo de Regresión Lineal 

In [ ]:
sns.lmplot(data=df, x="x", y="y")